In [2]:
from typing_extensions import TypedDict, Literal

class PortfolioState(TypedDict):
    amount_usd: float
    total_usd: float
    target_currency: Literal['INR', 'EUR']
    total: float

In [4]:
def calc_total(state: PortfolioState) -> PortfolioState:
    state['total_usd'] = state['amount_usd']*1.08
    return state

def convert_to_inr(state: PortfolioState) -> PortfolioState:
    state['total'] = state['total_usd']*91.0
    return state

def convert_to_eur(state: PortfolioState) -> PortfolioState:
    state['total'] = state['total_usd']*0.9
    return state

def choose_conversion(state: PortfolioState) -> str:
    return state['target_currency']

In [6]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(PortfolioState)

builder.add_node("calc_total_node", calc_total)
builder.add_node("convert_to_inr_node", convert_to_inr)
builder.add_node("convert_to_eur_node", convert_to_eur)

builder.add_edge(START, "calc_total_node")
builder.add_conditional_edges(
    "calc_total_node",
    choose_conversion,
    {
        'INR': "convert_to_inr_node",
        'EUR': "convert_to_eur_node"
    }
)
builder.add_edge("convert_to_inr_node", END)
# builder.add_edge("convert_to_inr_node","convert_to_eur_node", END)
builder.add_edge("convert_to_eur_node", END)
graph = builder.compile()

In [7]:
from IPython.display import Markdown
Markdown(f"```mermaid\n{graph.get_graph().draw_mermaid()}\n```")

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	calc_total_node(calc_total_node)
	convert_to_inr_node(convert_to_inr_node)
	convert_to_eur_node(convert_to_eur_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> calc_total_node;
	calc_total_node -. &nbsp;EUR&nbsp; .-> convert_to_eur_node;
	calc_total_node -. &nbsp;INR&nbsp; .-> convert_to_inr_node;
	convert_to_eur_node --> __end__;
	convert_to_inr_node --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [8]:
graph.invoke({
    'amount_usd': 1000.0,
    'target_currency': 'INR'
})

{'amount_usd': 1000.0,
 'total_usd': 1080.0,
 'target_currency': 'INR',
 'total': 98280.0}

In [9]:
graph.invoke({
    'amount_usd': 1000.0,
    'target_currency': 'EUR'
})

{'amount_usd': 1000.0,
 'total_usd': 1080.0,
 'target_currency': 'EUR',
 'total': 972.0}